In [1]:
import numpy as np
import pandas as pd
TRAIN_CSV_NAME = 'train_data.csv'
TRAIN_IMG_DIRECTORY = 'train_posters'
SEED = 1
np.random.seed(SEED) # Seed our randomness for reproducibilty

Load data

In [2]:
train_data_csv = pd.read_csv(TRAIN_CSV_NAME).values
genres = np.array(train_data_csv[:,-1]).astype(int)

In [3]:
# Choose indices for our test data
testIdx = np.random.choice(train_data_csv.shape[0], 200, replace=False)

In [4]:
y_test = genres[testIdx]
y_train = np.delete(genres, testIdx)

Load images

In [5]:
from bg.util import preprocess_images
images = np.array(preprocess_images(train_data_csv, TRAIN_IMG_DIRECTORY))

In [6]:
x_test = images[testIdx]
x_train = np.delete(images, testIdx, axis=0)

Build CNN

In [7]:
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential

Using TensorFlow backend.


In [8]:
img_x, img_y = 182, 268
input_shape = (img_x, img_y, 3)
num_classes= 4
batch_size = 100
epochs = 10000

# reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
# because the MNIST is greyscale, we only have a single channel - RGB colour images would have 3
x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 3)
x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 3)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices - this is for use in the
# categorical_crossentropy loss below
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (2894, 182, 268, 3)
2894 train samples
200 test samples


In [9]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [10]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [11]:
import keras.backend.tensorflow_backend as ktf

def get_session(gpu_fraction=.8):
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction,
                                allow_growth=True)
    return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


ktf.set_session(get_session())

In [ ]:
import matplotlib.pyplot as plt
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
plt.plot(range(1, 11), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

Train on 2894 samples, validate on 200 samples
Epoch 1/10000
2894/2894 [==============================] - 11s 4ms/step - loss: 2.9773 - acc: 0.2809 - val_loss: 1.3748 - val_acc: 0.3750
Epoch 2/10000
2894/2894 [==============================] - 4s 1ms/step - loss: 1.3475 - acc: 0.3614 - val_loss: 1.3741 - val_acc: 0.3700
Epoch 3/10000
2894/2894 [==============================] - 4s 1ms/step - loss: 1.2822 - acc: 0.3908 - val_loss: 1.3600 - val_acc: 0.3800
Epoch 4/10000
2894/2894 [==============================] - 4s 1ms/step - loss: 1.1764 - acc: 0.4603 - val_loss: 1.3590 - val_acc: 0.4200
Epoch 5/10000
2894/2894 [==============================] - 4s 1ms/step - loss: 1.0101 - acc: 0.5605 - val_loss: 1.6301 - val_acc: 0.3900
Epoch 6/10000
2894/2894 [==============================] - 4s 1ms/step - loss: 0.7998 - acc: 0.6638 - val_loss: 1.7123 - val_acc: 0.3550
Epoch 7/10000
2894/2894 [==============================] - 4s 1ms/step - loss: 0.5886 - acc: 0.7581 - val_loss: 2.0175 - val_acc: 